# Production simulation of a Molten NaCl Salt with OpenMM

The settings are practically identical to `exploration.ipynb`, the only differences are:

- Restart from the last snapshot of the exploration.
- The environment variable `REREP_NBARGS` is used to set the seed for the random initialization
  of the thermostat and the velocities.


In [ ]:
import json
import os

import mdtraj
import nglview
from bhmtf import add_nacl_forces, load_nacl
from openmm import unit
from stepup.core.api import amend
from utils import make_plots, runmd

In [ ]:
# Determine the random seed
arguments = {"seed": 42}
nbargs = os.getenv("REPREP_NBARGS")
if nbargs is not None:
    arguments.update(json.loads(nbargs))
seed = arguments["seed"]
prefix = f"prod{seed:04d}"

In [ ]:
# Inform StepUp of the output files it should expect, and which files are used as inputs.
amend(
    inp=[
        "bhmtf.py",
        "utils.py",
        "output/exploration_nve_final.pdb",
    ],
    out=[
        f"output/prod{seed:04d}_npt_final.pdb",
        f"output/prod{seed:04d}_npt_scalars.csv",
        f"output/prod{seed:04d}_npt_traj.dcd",
        f"output/prod{seed:04d}_npt_traj.npz",
        f"output/prod{seed:04d}_nve_final.pdb",
        f"output/prod{seed:04d}_nve_scalars.csv",
        f"output/prod{seed:04d}_nve_traj.dcd",
        f"output/prod{seed:04d}_nve_traj.npz",
    ],
)

## Simulation settings and initial state

In [ ]:
# Physical parameters
temperature = 1100 * unit.kelvin
pressure = 1 * unit.bar

# Time-related settings
timestep = 5 * unit.femtosecond
stride = 4
nstep_npt = 10000
tau_thermostat = 1 * unit.picosecond
nstep_nve = 12000

# Load the final structure of the exploration run.
system, topology, atnums, atcoords_init = load_nacl("output/exploration_nve_final.pdb")
add_nacl_forces(system, topology, do_charge=True, cutoff=1.5 * unit.nanometer)

## Re-equilibration in the NPT ensemble

In [ ]:
atcoords_equi, atvels_equi = runmd(
    f"{prefix}_npt",
    system,
    topology,
    atcoords_init,
    nstep=nstep_npt,
    timestep=timestep,
    stride=stride,
    temperature=temperature,
    pressure=pressure,
    seed=seed,
)

In [ ]:
traj = mdtraj.load(f"output/{prefix}_npt_traj.dcd", top="output/exploration_init.pdb")
view = nglview.show_mdtraj(traj)
view.add_unitcell()
view

In [ ]:
make_plots(f"{prefix}_npt")

## Production in the NVE ensemble

In [ ]:
atcoords_end = runmd(
    f"{prefix}_nve",
    system,
    topology,
    atcoords_equi,
    nstep=nstep_nve,
    timestep=timestep,
    stride=stride,
    atvels=atvels_equi,
)

In [ ]:
traj = mdtraj.load(f"output/{prefix}_nve_traj.dcd", top="output/exploration_init.pdb")
view = nglview.show_mdtraj(traj)
view.add_unitcell()
view

In [ ]:
make_plots(f"{prefix}_nve")